## Bibliotecas

In [1]:
import pandas as pd                             # Dataframes y utilidades
from spacy.lang.es.stop_words import STOP_WORDS # Palabras vacias
import spacy                                    # Procesamiento de lenguaje natural
import pickle                                   # Persistencia de objetos

## Cargar Corpus

In [2]:
corpus = pd.read_excel('../apoyo/Rest_Mex_2022.xlsx')
corpus.head()

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,1,Restaurant
2,Mala relación calidad-precio,seguiré corta y simple: limpieza\n- bad. Tengo...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,Al reservar un hotel con multipropiedad Mayan ...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"No pierdan su tiempo ni dinero, venimos porque...",1,Hotel


## Proceso de normalización

In [8]:
# Convertir las columnas datetime a strings
corpus['Title'] = corpus['Title'].astype(str)
corpus['Opinion'] = corpus['Opinion'].astype(str)

# Concatenar las dos columnas
corpus['texto'] = corpus['Title'] + ' ' + corpus['Opinion']

# Eliminar las columnas 'Title' y 'Opinion'
corpus = corpus.drop(['Title', 'Opinion'], axis=1)

# Verificar el DataFrame después de la eliminación
corpus.head()


,Polarity,Attraction,texto
0,1,Hotel,Pésimo lugar Piensen dos veces antes de ir a e...
1,1,Restaurant,No vayas a lugar de Eddie Cuatro de nosotros f...
2,1,Hotel,Mala relación calidad-precio seguiré corta y s...
3,1,Hotel,Minusválido? ¡No te alojes aquí! Al reservar u...
4,1,Hotel,Es una porqueria no pierdan su tiempo No pierd...


In [4]:
# --> Función para normalizar texto

# Cargar modelo de lenguaje español
nlp = spacy.load('es_core_news_sm')

def procesar_texto(texto):
    try:
        # Tokenización
        doc = nlp(texto)
        tokens = [token.text for token in doc]
        texto_tokenizado = " ".join(tokens)

        # Lematización
        doc = nlp(texto_tokenizado)
        lemas = [token.lemma_ for token in doc]
        texto_lematizado = " ".join(lemas)

        # Eliminación de stopwords
        doc = nlp(texto_lematizado)
        tokens_filtrados = [token.text for token in doc if token.pos_ not in ["DET", "ADP", "CCONJ", "PRON"]]
        texto_filtrado = " ".join(tokens_filtrados)
    except:
        # Ignorar contenido si no se puede procesar
        texto_filtrado = ""

    return texto_filtrado

In [9]:
# --> Normalizar contenido
lista_contenido = []

for i, contenido in enumerate(corpus['texto']):
    lista_contenido.append(procesar_texto(contenido))   # Normalizar contenido
    print(f"\rProgreso: {i/len(corpus):.1%}", end="")   # Mostrar porcentaje de avance

Progreso: 100.0%

In [10]:
print(corpus['texto'][0])
print("--------------------------------------------------")
print(lista_contenido[0])

Pésimo lugar Piensen dos veces antes de ir a este hotel, te molestan por todo, mi hijo de seis años traía un inflable de tamaño pequeño y nos decían que era muy grande ahí lo veían llorando y aún así seguían molestando, traíamos una hielera que parece bolsa nada de grande o con ruedas o de ese estilo y Tmb molestándonos las bebidas carísimas y ni agua te dejan tener en la alberca y no hay ni huéspedes en el hotel esto es en el mayan palace el grand mayan pude ser un poco mejor pero la verdad jamás vuelvo a venir a este lugar, te invitan a una cena show de mil pesos por persona y la cena es hoy dogs, las habitaciones nada fuera de lo normal
--------------------------------------------------
pésimo lugar pensar dos vez antes ir hotel , molestar , hijo seis año traer inflable tamaño pequeño decir que ser mucho grande ahí ver llorar aún así seguir molestar , traíir hielera que parecer bolso nada grande rueda estilo Tmb molestándono bebida carísimo agua dejar tener alberca no haber huésped 

## Guardar datos

In [12]:
# --> Guardar lista de contenido normalizado
with open('lista_contenido_normalizado.pkl', 'wb') as archivo:
    pickle.dump(lista_contenido, archivo)